In [1]:
import tempfile

import pandas as pd
import geopandas as gpd
import pyogrio
import httpx

In [2]:
with tempfile.TemporaryDirectory() as tmpdirname:
    with open(tmpdirname + "/data.csv", "wb") as f:
        r = httpx.get("https://www.czso.cz/documents/62353418/205988586/sldb2021_vzdelani_vek2_pohlavi.csv/5d7a6d5c-a7b1-468f-aa48-80560fbce267?version=1.1")
        f.write(r.content)
    edu_age_sex = pd.read_csv(tmpdirname + "/data.csv")

In [41]:
edu_age_sex["pohlavi_txt"] = edu_age_sex["pohlavi_txt"].fillna("both")

In [42]:
obce = edu_age_sex[edu_age_sex.uzemi_typ == "obec"]

In [43]:
vals = obce.set_index(["uzemi_kod", "vzdelani_txt", "vek_txt", "pohlavi_txt"])["hodnota"]
vals

uzemi_kod  vzdelani_txt  vek_txt        pohlavi_txt
500011     Bez vzdělání  15 - 64 let    both            6
                                        muž             4
                                        žena            2
                         65 a více let  both            3
                                        muž             2
                                                       ..
599999     Nezjištěno    15 - 64 let    muž            38
                                        žena           13
                         65 a více let  both           21
                                        muž            10
                                        žena           11
Name: hodnota, Length: 525336, dtype: int64

In [44]:
total_population = vals[:, :, :, "both"].groupby(level=0).sum()
by_edu = vals[:, :, :, "both"].groupby(level=[0, 1]).sum().unstack()

In [45]:
proportions = by_edu.div(total_population, axis=0) * 100

In [46]:
uzsz_url = "https://vdp.cuzk.cz/vymenny_format/soucasna/20230930_ST_UZSZ.xml.zip"

In [47]:
pyogrio.list_layers(uzsz_url)

array([['Staty', 'Point'],
       ['RegionySoudrznosti', 'Point'],
       ['Kraje', 'Point'],
       ['Vusc', 'Point'],
       ['Okresy', 'Point'],
       ['Orp', 'MultiPoint'],
       ['Pou', 'MultiPoint'],
       ['Obce', 'MultiPoint'],
       ['SpravniObvody', 'Point'],
       ['Mop', 'Point'],
       ['Momc', 'Point'],
       ['CastiObci', 'Point'],
       ['KatastralniUzemi', 'MultiPoint'],
       ['Zsj', 'MultiPoint']], dtype=object)

In [48]:
obce = gpd.read_file(uzsz_url, layer="Obce")

In [49]:
obec_to_okres = obce.set_index("Kod")["OkresKod"].rename_axis(index="uzemi_kod")
obec_to_okres

uzemi_kod
569151    3601.0
569291    3601.0
569330    3811.0
569402    3601.0
569615    3601.0
           ...  
551503    3301.0
530166    3201.0
548235    3304.0
550159    3306.0
530611    3201.0
Name: OkresKod, Length: 6258, dtype: float64

In [50]:
okresy = gpd.read_file(uzsz_url, layer="Okresy")

In [51]:
obec_to_okres = obec_to_okres.map(dict(zip(okresy.Kod, okresy.Nazev))).fillna("Praha")
obec_to_okres

uzemi_kod
569151      Havlíčkův Brod
569291      Havlíčkův Brod
569330             Jeseník
569402      Havlíčkův Brod
569615      Havlíčkův Brod
                ...       
551503    České Budějovice
530166             Benešov
548235           Pelhřimov
550159          Prachatice
530611             Benešov
Name: OkresKod, Length: 6258, dtype: object

In [52]:
proportions_w_okres = proportions.assign(okres=obec_to_okres)

In [53]:
proportions_w_okres.columns

Index(['Bez vzdělání', 'Nezjištěno', 'Neúplné základní vzdělání',
       'Nižší střední a střední vzdělání', 'Nástavbové vzdělání',
       'Pomaturitní studium', 'Vysokoškolské bakalářské vzdělání',
       'Vysokoškolské doktorské vzdělání',
       'Vysokoškolské magisterské vzdělání', 'Vyšší odborné vzdělání',
       'Vyšší odborné vzdělání v konzervatoři', 'Základní vzdělání',
       'Úplné střední odborné vzdělání', 'Úplné střední všeobecné vzdělání',
       'okres'],
      dtype='object', name='vzdelani_txt')

In [55]:
proportions_w_okres.columns = [
    'Without education',
    'Undetermined',
    'Incomplete primary education',
    'Lower secondary and secondary education',
    'Further education',
    'Post_maturita studies',
    'Bachelors degree',
    'Doctoral degree',
    'Masters degree',
    'Higher vocational education',
    'Higher vocational education in a conservatory',
    'Primary education',
    'Complete secondary vocational education',
    'Complete secondary general education',
    'Okres',
]
proportions_w_okres.columns = proportions_w_okres.columns.str.lower().str.replace(" ", "_")

In [56]:
proportions_w_okres

,without_education,undetermined,incomplete_primary_education,lower_secondary_and_secondary_education,further_education,post_maturita_studies,bachelors_degree,doctoral_degree,masters_degree,higher_vocational_education,higher_vocational_education_in_a_conservatory,primary_education,complete_secondary_vocational_education,complete_secondary_general_education,okres
uzemi_kod,,,,,,,,,,,,,,,
500011,0.570704,3.741281,1.141408,34.242232,1.775523,0.507292,2.853519,0.634115,12.935954,1.395054,0.126823,11.350666,17.945466,10.779962,Zlín
500020,0.885827,3.346457,1.968504,40.157480,2.066929,0.885827,1.771654,0.492126,6.299213,1.574803,0.000000,15.059055,16.338583,9.153543,Šumperk
500046,0.359195,3.232759,0.790230,39.152299,2.514368,0.790230,3.520115,0.215517,10.632184,1.364943,0.143678,9.770115,15.301724,12.212644,Nový Jičín
500062,0.238237,3.573556,1.072067,32.757594,2.084574,1.131626,3.037522,0.178678,13.281715,0.714711,0.119119,11.316260,18.701608,11.792734,Vsetín
500071,0.412939,2.890571,1.238816,34.067447,1.720578,0.757054,3.028217,0.137646,11.080523,0.894701,0.000000,9.772884,20.027529,13.971094,Vsetín
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599930,0.717703,5.645933,1.339713,38.133971,1.818182,0.622010,3.157895,0.191388,8.086124,1.148325,0.095694,11.531100,16.555024,10.956938,Nový Jičín
599948,0.714031,4.534095,1.677972,35.201714,1.856480,0.892538,3.498750,0.392717,11.602999,1.392360,0.071403,10.888968,16.137094,11.138879,Nový Jičín
599956,0.272480,3.337875,1.294278,35.217984,2.520436,0.681199,3.269755,0.272480,10.967302,1.430518,0.204360,10.354223,18.528610,11.648501,Nový Jičín


In [57]:
proportions_w_okres.to_csv("../../regression/data/education.csv")